# Medical insurance prediction

**Goal:** Predict the insurance costs using all available features. 

Credits: Data freely available from https://github.com/stedy/Machine-Learning-with-R-datasets

Let's load the dataset and the needed libraries

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
if 'google.colab' in str(get_ipython()):
  from google_drive_downloader import GoogleDriveDownloader as gdd
  gdd.download_file_from_google_drive(file_id='1xhM2x8WoIk3JicauwfQrhKmC1_kZ6KzX',
  dest_path='./insurance.csv')
else:
  print('Please put the file insurance.csv in the same folder as the notebook')

In [ ]:
data_original = pd.read_csv('./insurance.csv')

Let's look at the data

In [ ]:
data_original.head()

So we have seven features:
- age: age of primary beneficiary
- sex: insurance contractor gender, female, male
- bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9
- children: Number of children covered by health insurance / Number of dependents
- smoker: yes or no
- region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
- charges: Individual medical costs billed by health insurance. That's what we want to predict !

Remember that our goal is to predict the `charges` using all the other features. So `charges` represents our vector of `y `and all other features our matrix `X`

*Question*: Is it a regression or a classification problem ?

The first thing to do is to look if there are `Nan`values, namely errors or missing values.

In [ ]:
data_original.isnull().sum()

Good news ! No `Nan` values

Now let's have a look at the distribution of the different features

In [ ]:
n_bins = 20
fig, axs = plt.subplots(2, 4, sharey=False, tight_layout=True, figsize=(15, 5))

axs[0, 0].hist(data_original.age,bins=50)
axs[0, 0].set_title('age')
axs[0, 1].hist(data_original.sex)
axs[0, 1].set_title('sex')
axs[0, 2].hist(data_original.bmi,bins=50)
axs[0, 2].set_title('bmi')
axs[0, 3].hist(data_original.children)
axs[0, 3].set_title('children')
axs[1, 0].hist(data_original.smoker)
axs[1, 0].set_title('smoker')
axs[1, 1].hist(data_original.region)
axs[1, 1].set_title('region')
axs[1, 2].hist(data_original.charges,bins=50)
axs[1, 2].set_title('charges')

We can already notice that:
- distribution of age is quite uniform
- male and female are balanced
- bmi looks like a Gaussian distribution (good for stats)
- There are definitely more people without children or with only one
- There are definitely more people who don't smoke
- regions are balanced
- charges is a right skewed distribution -> we can use a log to make it normal ! Easy to use in practice, many models and statistical test make the normal distribtion hypothesis

It seems that we have both numerical and categorical features. Let's check that !

In [ ]:
data_original.info()

OK. we have 3 categorical variables. Let's convert them

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

data = data_original.copy()

# Sex
data.sex[data.sex=='female']=0
data.sex[data.sex=='male']=1
data.sex = data.sex.astype(str).astype(int)

# same as
#le = LabelEncoder()
#data.sex = le.fit_transform(data.sex)

# Smoker
# here it's better to do it by hand to be sure that 'no' is 0 and 'yes' is 1
data.smoker[data.smoker=='no']=0
data.smoker[data.smoker=='yes']=1
data.smoker = data.smoker.astype(str).astype(int)

# Region
enc = OneHotEncoder(sparse=False) # better not to use sparse matrix here (more interpretable)
cat_regions = data.region.unique() # list of regions
# OneHotEncoder needs a 2D array, so we need to transform it in a column vector 
# and in order to do that we need to have a numpy array, thus the .values
region = enc.fit_transform(data.region.values.reshape(-1, 1))

# region is now a numpy array, needs to transofmr in a panda frame
region_df = pd.DataFrame(region, columns=cat_regions)

# we can now drop the column with the old 'region' feature ...
data.drop('region',inplace=True, axis=1)

# ... and add the new binary variables resulting from OneHotEncoder
dataOHE = pd.concat([data, region_df], axis=1)


Let's have a look at the new DataFrame

In [ ]:
dataOHE.info()

OK. All variables are numerical. We can now start the analysis. It's always good to start by checking the correlations between the variables

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = dataOHE.corr()
sns.heatmap(corr,square=True,ax=ax)

We can see that:
- bmi seem to be a bit correlated with the nortwest region and with charges
- all the other high correlations concern `charges`

To see possible (direct) colinearities among features the scatter plot is also a good tool

In [ ]:
axeslist = pd.plotting.scatter_matrix(dataOHE, diagonal='hist', figsize=(10,10))

We can see that there is a clear pattern, like three different cluster, between age and charges. Let's have a closer look

In [ ]:
fig1 = plt.figure(figsize=(6, 6))
ax = plt.gca()
plt.scatter(dataOHE.age, dataOHE.charges, c='b',s=80, marker='o')
plt.ylabel('charges')
plt.xlabel('age')

We'll come back to that later.
Now let's have a closer look to the correlations of `charges`

In [ ]:
dataOHE.corr()['charges'].sort_values()

Well, guess what... it seems that if you smoke, your charges are higher. 
What happens if we change the distribution of charges using the log function ?

In [ ]:
charge_dist = dataOHE["charges"].values
logcharge = np.log(dataOHE["charges"].values)

In [ ]:
n_bins = 20
fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

axs[0].hist(charge_dist, bins=n_bins)
axs[0].set_title('charges')
axs[1].hist(logcharge, bins=n_bins)
axs[1].set_title('ln(charges)')

In [ ]:
dataOHE.charges=np.log(dataOHE.charges)
dataOHE.corr()['charges'].sort_values()

Well, now it seems that charges is also correalated a bit with the age, which is quite expected I would say

Let's look at the distribution of charges for smokers and non smokers

In [ ]:
f= plt.figure(figsize=(12,5))

ax=f.add_subplot(121)
sns.distplot(dataOHE[(dataOHE.smoker == 1)]["charges"],color='c',ax=ax)
ax.set_title('Distribution of charges for smokers')

ax=f.add_subplot(122)
sns.distplot(dataOHE[(dataOHE.smoker == 0)]['charges'],color='b',ax=ax)
ax.set_title('Distribution of charges for non-smokers')

We can see that non-smokers pay less but let's look if we also condition on the age

In [ ]:
g = sns.jointplot(x='age', y='charges',data=dataOHE[dataOHE.smoker==0], color="m")
g.set_axis_labels("$age$", "$charges$")
ax.set_title('Distribution of charges and age for non-smokers')

Well, it seems that for non-smokers charges increase with age. Quite expected !
Let's look now at smokers...

In [ ]:
g = sns.jointplot(x='age', y='charges',data=dataOHE[(dataOHE.smoker==1)], color="m")
g.set_axis_labels("$age$", "$charges$")
ax.set_title('Distribution of charges and age for smokers')

We can see that there are clearly two trends... might it be the sex ? Let's have a look

In [ ]:
g = sns.jointplot(x='age', y='charges',data=dataOHE[(dataOHE.smoker==1) & (dataOHE.sex==0)], color="m")
g.set_axis_labels("$age$", "$charges$")
ax.set_title('Distribution of charges and age for female smokers')

Nope, wrong hypothesis... let's try with bmi.

In [ ]:
fig1 = plt.figure(figsize=(6, 6))
ax = plt.gca()

plt.scatter(dataOHE[(dataOHE.smoker==1) & (dataOHE.bmi<30)].age, dataOHE[(dataOHE.smoker==1) & (dataOHE.bmi<30)].charges,
                c='b',s=80, marker='o',label='smoking thin')
plt.scatter(dataOHE[(dataOHE.smoker==1) & (dataOHE.bmi>=30)].age, dataOHE[(dataOHE.smoker==1) & (dataOHE.bmi>=30)].charges,
                c='r',s=80, marker='x',label='smoking obese')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.ylabel('charges')
plt.xlabel('age')

Bingo ! We can clearly see that, among smoker, obese people pay more than thin people regardless the age

And what if we also consider the non-smoking ones ?

In [ ]:
fig1 = plt.figure(figsize=(9, 9))
ax = plt.gca()

plt.scatter(dataOHE[(dataOHE.smoker==1) & (dataOHE.bmi<30)].age, dataOHE[(dataOHE.smoker==1) & (dataOHE.bmi<30)].charges,
                c='b',s=80, marker='o',label='smoking thin')
plt.scatter(dataOHE[(dataOHE.smoker==1) & (dataOHE.bmi>=30)].age, dataOHE[(dataOHE.smoker==1) & (dataOHE.bmi>=30)].charges,
                c='r',s=80, marker='x',label='smoking obese')
plt.scatter(dataOHE[(dataOHE.smoker==0) & (dataOHE.bmi<30)].age, dataOHE[(dataOHE.smoker==0) & (dataOHE.bmi<30)].charges,
                c='g',s=80, marker='s',label='non smoking thin')
plt.scatter(dataOHE[(dataOHE.smoker==0) & (dataOHE.bmi>=30)].age, dataOHE[(dataOHE.smoker==0) & (dataOHE.bmi>=30)].charges,
                c='y',s=80, marker='p',label='non smoking obese')

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.ylabel('charges')
plt.xlabel('age')

This was not expected ! It seems that if you do not smoke your charges will not change if you are obese or not. Glad to hear it (I am a non smoker..)

### ***Prediction***

Now it's time to predict the charges using the methods seen during the lectures

Let's first create our matrix of features `X`and target vector `y`

In [ ]:
y = dataOHE.charges
X = dataOHE.drop(['charges'], axis = 1)
name_features=X.columns
print(X)

We can see that the features `age` and `bmi` have different ranges with respect to the other variables and need to be scaled

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

n_features = X_train.shape[1]
n_samples = X_train.shape[0]

print('Number training samples ', n_samples, ' and number features', n_features)

scaler = StandardScaler()
scaler.fit(X_train[['age','bmi']])
X_train[['age','bmi']] = scaler.transform(X_train[['age','bmi']])
X_test[['age','bmi']] = scaler.transform(X_test[['age','bmi']])

# remember to always scale on the train and apply the same scaling factor to the test
# you should never use the test set before prediction


Let's try to use the three linear methods seen in the lecture: linear, Ridge and Lasso.

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# data are already centred and normalized
linear = LinearRegression(normalize=False,fit_intercept=False)
linear.fit(X_train, y_train)
print('Training score: ',linear.score(X_train, y_train), '; Test score: ', linear.score(X_test, y_test))

ridge = Ridge(normalize=False,fit_intercept=False)
ridge.fit(X_train, y_train)
print(ridge.score(X_test, y_test))

lasso = Lasso(alpha=1e-3, normalize=False,fit_intercept=False)
lasso.fit(X_train, y_train)
print(lasso.score(X_test, y_test))


It seems that results are not so different. We could use the function `enet_path` to check how parameter weights are changing when using a L2-norm regularization (Ridge) or a L1-norm (Lasso) or both (E-net).

In [ ]:
from sklearn.linear_model import enet_path

def enet_plot(l1_ratio,X_train,y_train,lambda_lasso,name_features):
    """Function plotting enet_path for some tuning parameter."""
    plt.rcParams['mathtext.fontset'] = 'cm'
    _, beta_enet, _ = enet_path(X_train, y_train, alphas=lambda_lasso, fit_intercept=False,
                                 l1_ratio=l1_ratio, return_models=False)
    fig1 = plt.figure(figsize=(15, 5))
    ax1 = fig1.add_subplot(111)
    ax1.plot(lambda_lasso, np.transpose(beta_enet), linewidth=3)
    plt.title("Enet path: " + r"$p={0}, n={1}, \gamma={2}$".format(n_features,
                                                        n_samples, l1_ratio), fontsize=16)

    plt.legend(name_features,bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.set_xscale('log')
    ax1.set_xlabel(r"$\lambda$")
    ax1.set_ylabel("Coefficient value")
    
    return beta_enet



In [ ]:
#range of lambda values
lambda_lasso = np.logspace(np.log10(100), np.log10(1e-2), num=100)

In [ ]:
# Ridge (only L2-loss)
theta_enet001 = enet_plot(0.00001,X_train,y_train,lambda_lasso,name_features)

In [ ]:
# E-net (both L2-loss and L1-loss)
theta_enet05 = enet_plot(0.5,X_train,y_train,lambda_lasso,name_features)

In [ ]:
# Lasso (only L1-loss)
theta_enet099 = enet_plot(0.999,X_train,y_train,lambda_lasso,name_features)

It seems that `sex` and `children` are important factors for non-smoking people `smoker=0`and that the four regions have the same importance. What happens if we ignore the region features ? Try to look at that

In [ ]:
X_new_train=X_train.drop(['southwest','northwest','southeast','northeast'], axis = 1)
X_new_test=X_test.drop(['southwest','northwest','southeast','northeast'], axis = 1)

In [ ]:
# Lasso (only L1-loss)
theta_enet099 = enet_plot(0.999,X_new_train,y_train,lambda_lasso,X_new_train.columns)

`children`, `sex` and `smoker` seem to be more important than age.
Let's try to use other methods with Cross-validation. 

Remember that also during Cross-validation you should not use test data before the prediction. This means that you should re-fit Standard Scaler at each fold using only the training part and not the test part. In order to be able to do that, you could use a `pipeline`

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

# SVR linear
scaler = StandardScaler()
svr = SVR(kernel='linear')
# here we build the pipeline in the right order (first scaling and then fitting)
pipe = Pipeline(steps=[('scaler', scaler), ('svr', svr)])
# here we choose the hyperparameter of the model
parameteres = {'svr__C': [0.001,0.1,1,10]}
# here we use GridSearchCV to look for the best hyper-parameters
grid = GridSearchCV(pipe, param_grid=parameteres, cv=5)
grid.fit(X_train,y_train)
# once found the best hyper-parameters we compute the test score
print('CV score for linear SVR is :', grid.score(X_test,y_test))
print('Best hyperparameters: ',grid.best_params_)


# SVR linear
scaler = StandardScaler()
svr = SVR(kernel='rbf')
pipe = Pipeline(steps=[('scaler', scaler), ('svr', svr)])
parameteres = {'svr__C': [0.001,0.1,1,10], 'svr__gamma':[0.01,0.1,1,10]}
grid = GridSearchCV(pipe, param_grid=parameteres, cv=5)
grid.fit(X_train,y_train)
print('CV score for rbf SVR is :', grid.score(X_test,y_test))
print('Best hyperparameters: ',grid.best_params_)

# KNN
scaler = StandardScaler()
KNN = KNeighborsRegressor()
pipe = Pipeline(steps=[('scaler', scaler), ('KNN', KNN)])
parameteres = {'KNN__n_neighbors': [2,5,8,10,20]}
grid = GridSearchCV(pipe, param_grid=parameteres, cv=5)
grid.fit(X_train,y_train)
print('CV score for KNN is :', grid.score(X_test,y_test))
print('Best hyperparameters: ',grid.best_params_)

Let's try to use Decision Trees based methods.
For Decision trees, we will focus on two hyperparameters: 'min_samples_split', the minimum number of samples required to split an internal node and 'min_samples_leaf', the minimum number of samples required to be at a leaf node. Other hyperparameters exist. Please have a look at the website of scikit-learn

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

# Decision Trees
scaler = StandardScaler()
Tree = DecisionTreeRegressor()
pipe = Pipeline(steps=[('scaler', scaler), ('Tree', Tree)])
parameteres = {'Tree__min_samples_split': [1,2,5,10,15,20,25], 'Tree__min_samples_leaf': [1,2,5,10,15]}
grid = GridSearchCV(pipe, param_grid=parameteres, cv=5)
grid.fit(X_train,y_train)
print('CV score for decision tree is :', grid.score(X_test,y_test))
print('Best hyperparameters: ',grid.best_params_)


To plot decision trees, we can also use the *tree* library from scikit-learn.
More information at: https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html 

This makes decision trees highly interpretable. 
**Question**: which are the most important features ?

In [ ]:
from sklearn import tree

Tree = DecisionTreeRegressor(min_samples_split=2, min_samples_leaf=15)
Tree.fit(X,y)

fig = plt.figure(figsize=(25,20))
tree.plot_tree(Tree, feature_names = name_features, max_depth=3, filled = True)
#fig.savefig('imagename.png')


In [ ]:
# Bagging
scaler = StandardScaler()
treeBagging = BaggingRegressor()
pipe = Pipeline(steps=[('scaler', scaler), ('treeBagging', treeBagging)])
parameteres = {'treeBagging__n_estimators': [10,20,50], 'treeBagging__max_samples': [0.2,0.5,0.6,0.9]}
grid = GridSearchCV(pipe, param_grid=parameteres, cv=5)
grid.fit(X_train,y_train)
print('CV score for bagging with decision tree is :', grid.score(X_test,y_test))
print('Best hyperparameters: ',grid.best_params_)

# Random Forest
scaler = StandardScaler()
RF = RandomForestRegressor()
pipe = Pipeline(steps=[('scaler', scaler), ('RF', RF)])
parameteres = {'RF__n_estimators': [10,20,50], 'RF__min_samples_split': [2,5,10], 'RF__min_samples_leaf': [2,5,10],'RF__max_samples': [0.2,0.5,0.6]}
grid = GridSearchCV(pipe, param_grid=parameteres, cv=5)
grid.fit(X_train,y_train)
print('CV score for random forest is :', grid.score(X_test,y_test))
print('Best hyperparameters: ',grid.best_params_)

# Adaboost
scaler = StandardScaler()
adaboost = AdaBoostRegressor()
pipe = Pipeline(steps=[('scaler', scaler), ('adaboost', adaboost)])
parameteres = {'adaboost__n_estimators': [10,20,50]}
grid = GridSearchCV(pipe, param_grid=parameteres, cv=5)
grid.fit(X_train,y_train)
print('CV score for AdaBoost is :', grid.score(X_test,y_test))
print('Best hyperparameters: ',grid.best_params_)

We can use random Forest to compute importance of features

In [ ]:
RF = RandomForestRegressor(max_samples=0.6, min_samples_leaf=5, min_samples_split=5, n_estimators=50)
RF.fit(X,y)
importances = RF.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure(figsize=(15, 5))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
        color="r", align="center")
plt.xticks(range(X.shape[1]), X.columns[indices])
plt.xlim([-1, X.shape[1]])
plt.show()

We can see that the most important features are `smoker` and `age`. 

**Question**: it's still not clear whether the region features are important or not. And what about the number of children ? Try to answer these questions by squeezing a little bit more the data